# Weighted Dataset Model

- Train: AnyAgree_train
- Test: 75Agree_test

# TODO

- everything

https://chatgpt.com/c/683f09e6-2b90-800f-b478-2cc480d87d50

In [1]:
"""import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from transformers import get_scheduler
from sklearn.metrics import accuracy_score
from tqdm import tqdm

# Step 1: Define dataset
class WeightedTextDataset(Dataset):
    def __init__(self, texts, labels, weight_classes, tokenizer, max_len, weight_map):
        self.encodings = tokenizer(texts, truncation=True, padding=True, max_length=max_len)
        self.labels = labels
        self.weights = [weight_map[w] for w in weight_classes]

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        item['weights'] = torch.tensor(self.weights[idx], dtype=torch.float)
        return item

    def __len__(self):
        return len(self.labels)

# Step 2: Prepare tokenizer and model
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

# Optional GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Step 3: Dummy data
texts = ["bad service", "great experience", "not good", "excellent", "horrible wait"]
labels = [0, 1, 0, 1, 0]                          # true labels
ref_classes = [1, 2, 1, 2, 1]                     # weight-basis classes
weight_map = {1: 2.0, 2: 1.0}                     # ref_class => weight

# Step 4: Dataloaders
train_dataset = WeightedTextDataset(texts, labels, ref_classes, tokenizer, max_len=32, weight_map=weight_map)
train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True)

# Step 5: Optimizer and scheduler
optimizer = AdamW(model.parameters(), lr=2e-5)
lr_scheduler = get_scheduler("linear", optimizer=optimizer,
                             num_warmup_steps=0,
                             num_training_steps=len(train_loader)*3)

# Step 6: Training loop
model.train()
for epoch in range(3):
    for batch in tqdm(train_loader):
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(input_ids=batch["input_ids"],
                        attention_mask=batch["attention_mask"],
                        labels=batch["labels"])
        loss = outputs.loss
        logits = outputs.logits

        with torch.no_grad():
            per_sample_loss = torch.nn.functional.cross_entropy(
                logits, batch["labels"], reduction='none')
            weighted_loss = (per_sample_loss * batch["weights"]).mean()

        optimizer.zero_grad()
        weighted_loss.backward()
        optimizer.step()
        lr_scheduler.step()

# Step 7: Evaluation
model.eval()
all_preds = []
all_labels = []
with torch.no_grad():
    for batch in train_loader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(input_ids=batch["input_ids"],
                        attention_mask=batch["attention_mask"])
        preds = torch.argmax(outputs.logits, dim=-1)
        all_preds.extend(preds.cpu().tolist())
        all_labels.extend(batch["labels"].cpu().tolist())

acc = accuracy_score(all_labels, all_preds)
print("Accuracy:", acc)
"""
''

''